In [1]:
# Load data from MySQL to perform exploratory data analysis
import settings
import mysql.connector
import pandas as pd
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
df = pd.read_sql('SELECT * FROM {}'.format(settings.TABLE_NAME), con=db_connection)

In [2]:
df.head()

,id_str,datetime,text,polarity
0,1158456816558387202,2019-08-05 19:16:42,RT @ananavarro: Trump can speak out against:\n...,0
1,1158456816696623104,2019-08-05 19:16:42,"RT @GabbyGiffords: If you’re angry, we need to...",0
2,1158456816679866368,2019-08-05 19:16:42,RT @GlennKesslerWP: Mental illness pulled the ...,0
3,1158456816734494721,2019-08-05 19:16:42,RT @BillOReilly: It is beyond revolting that B...,0
4,1158456816659050496,2019-08-05 19:16:42,Putin a Trump: desarrollaremos nuevos misiles ...,0


In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
ax = sns.violinplot(x=df["polarity"])